Parte 1

Preprocesamientos no otorgados por keras


transfromación a tensores, entendibles por torch

In [2]:
import os
import pandas as pd
from PIL import Image
import torch
from torchvision import transforms
import csv
import string

In [3]:


# Transformaciones
transform_im = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normaliza a [-1, 1]
])


#Preprocesamiento para texto
# Diccionario de mapeo letra -> índice (0 a 51)
all_letters = list(string.ascii_uppercase + string.ascii_lowercase)
letter_to_idx = {char: idx for idx, char in enumerate(all_letters)}

In [4]:
# Listas para acumular los datos
img_list = []
label_values = []  # Cambiado el nombre para mayor claridad

CSV_PATH = "arial_dataset/arial_letters.csv"
IMG_DIR = "arial_dataset"

with open(CSV_PATH, "r", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Saltar encabezado

    for row in reader:
        img_name, label = row
        img_path = os.path.join(IMG_DIR, img_name)

        # Procesar imagen
        img = Image.open(img_path)
        img_list.append(transform_im(img))

        # Procesar etiqueta
        label_idx = letter_to_idx[label]
        label_values.append(label_idx)

# Convertir a tensores al final
img_tensor = torch.stack(img_list)  # Tensor de imágenes
text_tensor = torch.tensor(label_values, dtype=torch.long)  # Tensor de etiquetas numéricas


In [5]:
print("imgtensor: \n\n", img_tensor, "\n\n")
print("size im: n\n", img_tensor.shape, "\n\n")
print("Texttensor: \n\n",text_tensor)

imgtensor: 

 tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        ...,


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1.

Dividiendo los datos de entrenamiento y guardándolos

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(img_tensor, text_tensor, test_size=0.2, random_state=42)

# Guardar los tensores
torch.save({
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test
}, 'train_test_split.pt')


In [9]:
# Carga el contenido del archivo
data = torch.load("train_test_split.pt")

# Accede a los datos si están organizados en un diccionario
X_train = data["X_train"]
X_test = data["X_test"]
y_train = data["y_train"]
y_test = data["y_test"]





C:\Users\iagoc\AppData\Local\Temp\ipykernel_5392\737453814.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("train_test_split.pt")


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class LetterClassifier(nn.Module):
    def __init__(self, num_classes=26):
        super(LetterClassifier, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # 1x128x128 → 32x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 64x64x64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 64x32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# 128x32x32
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 128x16x16
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x  # logits (sin softmax, para usar con CrossEntropyLoss)


In [12]:
from torch.utils.data import DataLoader
# DataLoader procesamiento por batches, mezcla aleatoriamente los datos y carga los datos en paralelo
from torch.utils.data import TensorDataset 
# TensorDataset crea un dataset a partir de varios tensores

In [13]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)


batch_size = 32


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Definimos la función de pérdida y el optimizador

In [14]:
import torch.optim as optim

In [15]:
model = LetterClassifier(num_classes=52)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Iniciamos el modelo

In [17]:
import copy
import random
import string
import torch
import torch.optim as optim

# —————— Configuración inicial de tu red y datos ——————
# Asume que `model`, `criterion`, `train_loader` están definidos en tu entorno.
# Por ejemplo:
# model = MyNeuralNet()
# criterion = torch.nn.CrossEntropyLoss()
# train_loader = DataLoader(mi_dataset, batch_size=32)

# Diccionario para imprimir índices como letras
idx_to_letter = list(string.ascii_uppercase + string.ascii_lowercase)
def tensor_a_letras(tensor):
    return [idx_to_letter[idx.item()] for idx in tensor]

# —————— Función de evaluación ——————
# Recibe una configuración de hiperparámetros y devuelve el loss total entrenando el modelo clonado.
def evaluar_loss(config):
    # CLONAMOS modelo original para no modificarlo
    modelo_copia = copy.deepcopy(model)
    # DEFINIMOS optimizador Adam con los parámetros de 'config'
    optimizer = optim.Adam(
        modelo_copia.parameters(),
        lr=config['lr'],
        betas=config.get('betas', (0.9, 0.999)),
        eps=config.get('eps', 1e-8),
        weight_decay=config.get('weight_decay', 0.0),
        amsgrad=config.get('amsgrad', False)
    )
    num_epochs = config.get('num_epochs', 3)
    total_loss = 0.0

    modelo_copia.train()
    for _ in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = modelo_copia(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    return total_loss

# —————— Generador de vecindario ——————
def generar_vecindario(sol_actual):
    """
    Crea vecinos variando ligeramente el lr.
    Aquí solo variamos LR, pero podrías incluir betas, weight_decay, etc.
    """
    deltas = [-0.0005, -0.0001, 0.0001, 0.0005]
    vecinos = []
    for d in deltas:
        lr_nuevo = max(1e-6, sol_actual['lr'] + d)
        vecino = sol_actual.copy()
        vecino['lr'] = lr_nuevo
        vecinos.append(vecino)
    return vecinos

# —————— Tabu Search ——————
# Aquí implementamos:
#  • SHORT-TERM MEMORY: tabu_list evita revisitar movimientos prohibidos.
#  • LONG-TERM MEMORY: long_term registra estadísticas de exploración.
#  • ASPIRATION CRITERION: permite aceptar un movimiento tabu si mejora el mejor global.

def tabu_search(iters=20, tabu_size=5, initial_config=None):
    # 1) Configuración inicial
    if initial_config is None:
        current = {'lr': random.uniform(1e-4, 1e-2), 'num_epochs': 3}
    else:
        current = initial_config.copy()

    # Evaluar estado inicial
    best = current.copy()
    best_score = evaluar_loss(best)

    # SHORT-TERM MEMORY
    tabu_list = []  # movimientos prohibidos temporalmente

    # LONG-TERM MEMORY
    long_term = {
        'visits': {},      # cuántas veces se probó cada lr
        'best_scores': {}  # mejor loss por lr
    }
    lr0 = current['lr']
    long_term['visits'][lr0] = 1
    long_term['best_scores'][lr0] = best_score

    for i in range(iters):
        # 2) Generar vecindario reducido (Candidate List Strategies)
        #    Para reducir costo, sólo consideramos un subconjunto de vecinos.
        vecinos = generar_vecindario(current)[:4]  # ejemplo: sólo primeros 4

        # 3) Filtrado con Tabu y Criterio de Aspiración
        candidatos = []
        for v in vecinos:
            is_tabu = v in tabu_list
            score_v = evaluar_loss(v)  # pre-evaluamos breve para aspiración
            # Aspiration: si un movimiento tabu mejora el best_score global, aceptamos
            if not is_tabu or score_v < best_score:
                candidatos.append((score_v, v, is_tabu))
            # Actualizar LTM independientemente
            lr = v['lr']
            long_term['visits'][lr] = long_term['visits'].get(lr, 0) + 1
            prev_best = long_term['best_scores'].get(lr, float('inf'))
            long_term['best_scores'][lr] = min(prev_best, score_v)

        # 4) Seleccionar mejor candidato
        candidatos.sort(key=lambda x: x[0])
        best_vec_score, best_vec, was_tabu = candidatos[0]

        # 5) Actualizar estado
        current = best_vec
        # Si no se usó aspiración para este tabu, añadimos
        if not was_tabu:
            tabu_list.append(best_vec)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)

        # 6) Actualizar mejor global
        if best_vec_score < best_score:
            best_score = best_vec_score
            best = best_vec.copy()

        print(f"Iter {i+1}/{iters} | LR={current['lr']:.6f} | Loss={best_vec_score:.4f} | MejGlobal={best_score:.4f}")

    # 7) Devolver mejor configuración, su score y memoria a largo plazo
    return best, best_score, long_term

# —————— Ejecución ——————
if __name__ == "__main__":
    mejor_conf, mejor_loss, memoria = tabu_search(
        iters=15,
        tabu_size=4,
        initial_config={'lr': 0.001, 'num_epochs': 5}
    )
    print("\n=== Mejor configuración encontrada ===")
    print(mejor_conf)
    print(f"Loss acumulado = {mejor_loss:.4f}")
    print("\n=== Resumen de LONG-TERM MEMORY ===")
    print(memoria)



Iter 1/15 | LR=0.000500 | Loss=4.9515 | MejGlobal=4.9515
Iter 2/15 | LR=0.000400 | Loss=3.5501 | MejGlobal=3.5501
Iter 3/15 | LR=0.000500 | Loss=2.7955 | MejGlobal=2.7955
Iter 4/15 | LR=0.000400 | Loss=1.9177 | MejGlobal=1.9177
Iter 5/15 | LR=0.000300 | Loss=1.9122 | MejGlobal=1.9122
Iter 6/15 | LR=0.000200 | Loss=1.6822 | MejGlobal=1.6822
Iter 7/15 | LR=0.000100 | Loss=1.7181 | MejGlobal=1.6822
Iter 8/15 | LR=0.000600 | Loss=4.4605 | MejGlobal=1.6822
Iter 9/15 | LR=0.000100 | Loss=1.5817 | MejGlobal=1.5817
Iter 10/15 | LR=0.000200 | Loss=1.5691 | MejGlobal=1.5691
Iter 11/15 | LR=0.000300 | Loss=1.2457 | MejGlobal=1.2457
Iter 12/15 | LR=0.000200 | Loss=1.5643 | MejGlobal=1.2457
Iter 13/15 | LR=0.000100 | Loss=2.0158 | MejGlobal=1.2457
Iter 14/15 | LR=0.000600 | Loss=7.2393 | MejGlobal=1.2457
Iter 15/15 | LR=0.000100 | Loss=2.0696 | MejGlobal=1.2457

=== Mejor configuración encontrada ===
{'lr': 0.00030000000000000003, 'num_epochs': 5}
Loss acumulado = 1.2457

=== Resumen de LONG-TERM M

In [18]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()  # Muy importante: desactiva dropout/batchnorm si los hubiera
all_preds = []
all_labels = []

with torch.no_grad():  # No se calculan gradientes, más eficiente
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Métricas
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Accuracy en test: {acc * 100:.2f}%")

# Reporte más detallado
print("\n📋 Clasificación por clase:")
print(classification_report(all_labels, all_preds, target_names=list(string.ascii_uppercase)))



✅ Accuracy en test: 97.98%

📋 Clasificación por clase:
              precision    recall  f1-score   support

           A       1.00      0.98      0.99        50
           B       0.97      0.95      0.96        41
           C       1.00      0.93      0.97        45
           D       1.00      0.98      0.99        45
           E       1.00      0.95      0.97        38
           F       0.90      1.00      0.95        37
           G       1.00      0.87      0.93        38
           H       1.00      1.00      1.00        45
           I       1.00      1.00      1.00        41
           J       0.97      1.00      0.99        37
           K       1.00      1.00      1.00        26
           L       1.00      1.00      1.00        39
           M       0.97      1.00      0.99        37
           N       0.98      0.95      0.96        42
           O       0.95      0.97      0.96        40
           P       1.00      0.97      0.99        37
           Q       0.96  

In [19]:
torch.save(model.state_dict(), "letter_classifier.pth")

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
class LetterClassifier(nn.Module):
    def __init__(self, num_classes=52):
        super(LetterClassifier, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # 1x128x128 → 32x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 64x64x64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 64x32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# 128x32x32
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 128x16x16
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x  # logits (sin softmax, para usar con CrossEntropyLoss)

In [22]:
modelo = LetterClassifier()
modelo.load_state_dict(torch.load('letter_classifier.pth', map_location='cpu'))
modelo.eval()


C:\Users\iagoc\AppData\Local\Temp\ipykernel_5392\1852476355.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo.load_state_dict(torch.load('letter_classifier.pth', ma

LetterClassifier(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=32768, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=52, bias=True)
  )
)

In [23]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import string

# Paso 1: Diccionario para convertir índices a letras
idx_to_letter = list(string.ascii_uppercase + string.ascii_lowercase)

# Paso 2: Cargar y transformar la imagen
transform = transforms.Compose([
    transforms.Grayscale(),                  # Asegura 1 canal
    transforms.Resize((128, 128)),             # Tamaño esperado
    transforms.ToTensor(),                   # Convertir a tensor [0,1]
    transforms.Normalize((0.5,), (0.5,))     # Normalización opcional
])

# Paso 3: Cargar imagen
img_path = "arial_dataset/A_0.png"
imagen = Image.open(img_path)
imagen = transform(imagen)
imagen = imagen.unsqueeze(0)  # Añade dimensión de batch: [1, 1, 28, 28]

# Paso 4: Cargar el modelo ya corregido
modelo = LetterClassifier()
modelo.load_state_dict(torch.load("letter_classifier.pth", map_location='cpu'))
modelo.eval()

# Paso 5: Realizar predicción
with torch.no_grad():
    salida = modelo(imagen)
    prediccion = torch.argmax(salida, dim=1).item()
    letra = idx_to_letter[prediccion]

print(f"Letra predicha: {letra}")


Letra predicha: A


C:\Users\iagoc\AppData\Local\Temp\ipykernel_5392\2694318761.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo.load_state_dict(torch.load("letter_classifier.pth", m